In [1]:
! pip install --no-index --find-links /kaggle/input/pipstuff pyctcdecode
! pip install --no-index --find-links /kaggle/input/pipstuff bnunicodenormalizer

Looking in links: /kaggle/input/pipstuff
Processing /kaggle/input/pipstuff/pyctcdecode-0.5.0-py2.py3-none-any.whl
Processing /kaggle/input/pipstuff/pygtrie-2.5.0-py3-none-any.whl (from pyctcdecode)
Processing /kaggle/input/pipstuff/hypothesis-6.82.0-py3-none-any.whl (from pyctcdecode)
Looking in links: /kaggle/input/pipstuff
Processing /kaggle/input/pipstuff/bnunicodenormalizer-0.1.6-py3-none-any.whl


In [2]:
# !tar xvfz /kaggle/input/yellowking-dlsprint-inference/python-packages2/jiwer.tgz
# !tar xvfz /kaggle/input/yellowking-dlsprint-inference/jiwer/python-Levenshtein-0.12.2.tar.gz
# !pip install --no-index --find-links /kaggle/input/yellowking-dlsprint-inference/jiwer/ python-Levenshtein
# !pip install --no-index --find-links /kaggle/input/yellowking-dlsprint-inference/jiwer/ jiwer

In [3]:
import os
import torch
import transformers
from transformers import AutoTokenizer, AutoModel, AutoFeatureExtractor, AutoProcessor, pipeline
from glob import glob
import librosa
from torch.utils.data import DataLoader, Subset
import bnunicodenormalizer
from bnunicodenormalizer import Normalizer 
import pandas as pd
# import jiwer

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

cuda


In [4]:
# CHANGE ACCORDINGLY
BATCH_SIZE = 32
TEST_DIRECTORY = '/kaggle/input/bengaliai-speech/test_mp3s'
paths = glob(os.path.join(TEST_DIRECTORY,'*.mp3'))
print(paths)

['/kaggle/input/bengaliai-speech/test_mp3s/a9395e01ad21.mp3', '/kaggle/input/bengaliai-speech/test_mp3s/0f3dac00655e.mp3', '/kaggle/input/bengaliai-speech/test_mp3s/bf36ea8b718d.mp3']


In [5]:
def add_file_path(text):
  text = "/kaggle/input/bengaliai-speech/train_mp3s/" + text + '.mp3'
  return text

In [6]:
df = pd.read_csv('/kaggle/input/bengaliai-speech/train.csv')
df = pd.DataFrame(df, columns=['id', 'sentence'])
df['file'] = df['id'].map(lambda x: add_file_path(x))
print(df)

                  id                                           sentence  \
0       000005f3362c                              ও বলেছে আপনার ঠিকানা!   
1       00001dddd002                 কোন মহান রাষ্ট্রের নাগরিক হতে চাও?   
2       00001e0bc131     আমি তোমার কষ্টটা বুঝছি, কিন্তু এটা সঠিক পথ না।   
3       000024b3d810  নাচ শেষ হওয়ার পর সকলে শরীর ধুয়ে একসঙ্গে ভোজন...   
4       000028220ab3                                হুমম, ওহ হেই, দেখো।   
...              ...                                                ...   
963631  ffffd07108b7                           আপনার সাথে কথা বলতে চাই।   
963632  ffffde37678a  সুতরাং পরের দিন আর-একটা ছবি না লইয়া চিত্রকর ছা...   
963633  ffffe1b5f095  সামাজিক কর্মকাণ্ডসমিতিতে গিয়ে দেখা যায়, শিল্পী...   
963634  ffffec31636e  গুগল ম্যাপসের সাহায্যে খুঁজে পাওয়া যাবে কোন জা...   
963635  fffff9f38566                      তোমরা আমাকে কী নাম ধরে ডাকবে?   

                                                     file  
0       /kaggle/input/bengaliai-speech/

In [7]:
from datasets import Dataset, load_metric

train_data = Dataset.from_pandas(df)
train_data

Dataset({
    features: ['id', 'sentence', 'file'],
    num_rows: 963636
})

In [8]:
# def extract_all_chars(batch):
#   all_text = " ".join(batch["sentence"])
#   vocab = list(set(all_text))
#   return {"vocab": [vocab], "all_text": [all_text]}

# vocab_train = train_data.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=train_data.column_names)

In [9]:
# vocab_train

In [10]:
# vocab_list = list(set(vocab_train["vocab"][0]))

# vocab_dict = {v: k for k, v in enumerate(vocab_list)}
# print(vocab_dict)

In [11]:
# vocab_dict["|"] = vocab_dict[" "]
# del vocab_dict[" "]
# vocab_dict["[UNK]"] = len(vocab_dict)
# vocab_dict["[PAD]"] = len(vocab_dict)
# print(len(vocab_dict))

In [12]:
# import json
# with open('/kaggle/working/vocab.json', 'w') as vocab_file:
#     json.dump(vocab_dict, vocab_file)

In [13]:
# from transformers import Wav2Vec2CTCTokenizer

# tokenizer = Wav2Vec2CTCTokenizer("/kaggle/working/vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [14]:
# from transformers import Wav2Vec2FeatureExtractor

# feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [15]:
# from transformers import Wav2Vec2Processor

# processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [16]:
# processor.save_pretrained("/kaggle/working/")

In [17]:
# class MyASRDataset(torch.utils.data.Dataset):
#     def __init__(self, data, tokenizer, max_audio_length):
#         self.dataset = data
#         self.tokenizer = tokenizer
#         self.max_audio_length = max_audio_length

#     def __len__(self):
#         return len(self.dataset)

#     def __getitem__(self, idx):
#         item = self.dataset[idx]
#         file_path = item['file']
#         waveform, sample_rate = librosa.load(file_path, sr=16000)

#         transcription = item['sentence']
#         tokenized_inputs = self.tokenizer(transcription, padding='max_length', truncation=True, max_length=self.max_audio_length, return_tensors='pt')

#         processed_item = {
#             'input_ids': tokenized_inputs['input_ids'].squeeze(),
#             'attention_mask': tokenized_inputs['attention_mask'].squeeze(),
#             'labels': tokenized_inputs['input_ids'].squeeze(),  # Use 'input_ids' as labels for non-masked language modeling
#             'file': waveform
#         }
#         return processed_item


# max_audio_length = 10000
# asr_dataset = MyASRDataset(train_data, tokenizer, max_audio_length)

In [18]:
# wer_metric = load_metric('/kaggle/input/metrics/wer.py')

In [19]:
# class MyDataCollator:
#     def __call__(self, features):
#         # Merge lists of input_ids, attention_mask, and files
#         input_ids = [feature['input_ids'] for feature in features]
#         attention_mask = [feature['attention_mask'] for feature in features]
#         files = [feature['file'] for feature in features]
        
#         # Convert lists to tensors
#         input_ids = torch.stack(input_ids)
#         attention_mask = torch.stack(attention_mask)

#         return {
#             'input_ids': input_ids,
#             'attention_mask': attention_mask,
#             'file': files
#         }


# data_collator = MyDataCollator()

In [20]:
#INFERENCE
class CFG:
    model = AutoModel.from_pretrained('/kaggle/input/bengali-model/bengali/')
    tokenizer = AutoTokenizer.from_pretrained('/kaggle/input/bengali-model/bengali/')
    feature_extractor = AutoFeatureExtractor.from_pretrained('/kaggle/input/bengali-model/bengali/')
    processor = AutoProcessor.from_pretrained('/kaggle/input/bengali-model/bengali/')

/opt/conda/lib/python3.10/site-packages/transformers/configuration_utils.py:380: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of the model checkpoint at /kaggle/input/bengali-model/bengali/ were not used when initializing Wav2Vec2Model: ['lm_head.weight', 'lm_head.bias']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model

In [21]:
my_asrLM = pipeline("automatic-speech-recognition", model='/kaggle/input/bengali-model/bengali/', feature_extractor=CFG.processor.feature_extractor, tokenizer=CFG.processor.tokenizer, device=0)

In [22]:
speech, sr = librosa.load('/kaggle/input/bengaliai-speech/test_mp3s/0f3dac00655e.mp3', sr=CFG.feature_extractor.sampling_rate)

In [23]:
my_asrLM([speech]*2, chunk_length_s=112, stride_length_s=None)

[{'text': '৮াথান খমললাবুরাআখা াছাা'}, {'text': '৮াথান খমললাবুরাআখা াছাা'}]

In [24]:
my_asrLM

In [25]:
class AudioDataset():
    def __init__(self, paths):
        self.paths = paths
    def __len__(self):
        return len(self.paths)
    def __getitem__(self,idx):
        speech, sr = librosa.load(self.paths[idx], sr=CFG.feature_extractor.sampling_rate) 
#         print(speech.shape)
        return speech

In [26]:
dataset = AudioDataset(paths)
dataset[0]

array([-1.7763568e-14,  4.2632564e-14,  2.2737368e-13, ...,
       -6.2642025e-06,  2.5706822e-07, -5.0473864e-06], dtype=float32)

In [27]:
def collate_fn_padd(batch):
    '''
    Padds batch of variable length

    note: it converts things ToTensor manually here since the ToTensor transform
    assume it takes in images rather than arbitrary tensors.
    '''
    ## get sequence lengths
    lengths = torch.tensor([ t.shape[0] for t in batch ])
    ## padd
    batch = [ torch.Tensor(t) for t in batch ]
    batch = torch.nn.utils.rnn.pad_sequence(batch)
    ## compute mask
    mask = (batch != 0)
    return batch, lengths, mask



In [28]:
dataloader = DataLoader(dataset, batch_size=32, shuffle=False, num_workers=2, collate_fn=collate_fn_padd)

In [29]:
preds_all = []
for batch, lengths, mask in dataloader:
    preds = my_asrLM(list(batch.numpy().transpose()))
    preds_all+=preds

In [30]:
bnorm = Normalizer()
def normalize(sen):
    _words = [bnorm(word)['normalized']  for word in sen.split()]
    return " ".join([word for word in _words if word is not None])

def dari(sentence):
    try:
        if sentence[-1]!="।":
            sentence+="।"
    except:
        print(sentence)
    return sentence



In [31]:
df= pd.DataFrame(
    {
        "id":[p.split(os.sep)[-1].replace('.mp3','') for p in paths],
        "sentence":[p['text']for p in preds_all]
    }
)
df.sentence= df.sentence.apply(lambda x:normalize(x))
df.sentence= df.sentence.apply(lambda x:dari(x))

In [32]:
df

,id,sentence
0,a9395e01ad21,ত্রেই হই ভাকক্যেইবিক রইহি হতুক যের উর তুহাকর ত...
1,0f3dac00655e,আথান খলুলা লবুরাআথা ছাখা।
2,bf36ea8b718d,হারার যেইগিম্মা ফমুষো না থেকথার্ষএ হনা মারুভভ্...


In [33]:
df.to_csv("submission.csv", index=False)